<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">3. Summary of Pre-processing Methods</font><br>
</div>


>1. Read a csv file
>2. Get a subset of dataframe: 'Country_Region' & 'Confirmed'
>3. Clean messy column names 
    - e.g., 'Country_Region' vs 'Country/Region'
>4. Delete rows with missing values on 'Confirmed' 
>5. Change datatype for 'Confirmed'
    - float -> int
>6. Clean messy country names 
    - e.g., South Korea, Repulic of Korea
    - "apply" values from a json file
>7. Group by 'Country_Region'
    - Aggregate 'Province_State' data
>8. Change column name: 'Confirmed' -> 'File name'
    - e.g., 1/22/2020


<table>
<thead>
<tr>
<th>B</th>
<th>C</th>
<th>D</th>
<th>E</th>
<th>F</th>
<th>G</th>
<th>H</th>
</tr>
</thead>
<tbody>
<tr>
<th>Country_Region</th>
<th>Image URL</th>
<th>1/22/20</th>
<th>1/23/20</th>
<th>1/24/20</th>
<th>1/25/20</th>
<th>1/26/20</th>
</tr>
<tr>
<th>Afghanistan</th>
<th>https://www.countryflags.io/AO/flat/64.png</th>
<th>0</th>
<th>0</th>
<th>0</th>
<th>0</th>
<th>0</th>
</tr>
<tr>
<th>Albania</th>
<th>https://www.countryflags.io/BI/flat/64.png</th>
<th>0</th>
<th>0</th>
<th>0</th>
<th>0</th>
<th>0</th>
</tr>
<tr>
<th>Algeria</th>
<th>https://www.countryflags.io/BJ/flat/64.png</th>
<th>0</th>
<th>0</th>
<th>0</th>
<th>0</th>
<th>0</th>
</tr>
</tbody>
</table>

In [1]:
import json
import pandas as pd

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

def country_name_convert(arg_df):
    if arg_df['Country_Region'] in json_data:
        arg_df['Country_Region'] = json_data[arg_df['Country_Region']]
    return arg_df['Country_Region']

def create_dataframe(filename):
    PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')  #1. Read a csv file 
                                                              # filename: e.g., 01-22-2020.csv
    try:
        doc = doc[['Country_Region', 'Confirmed']]  #2. Get a subset: 'Country_Region' & 'Confirmed'
    except:
        doc = doc[['Country/Region', 'Confirmed']]  
        doc.columns = ['Country_Region', 'Confirmed'] #3. Clean messy column names
    doc = doc.dropna(subset=['Confirmed'])     #4. Delete rows with missing values
    doc = doc.astype({'Confirmed': 'int64'})   #5. Change datatype: float -> int
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)   #6. Clean messy country names using 'apply'
    doc = doc.groupby('Country_Region').sum()  #7. Group by 'Country_Region' 
                                               # 'Country_Region' now Index

    #8. Change column name: 'Confirmed' -> 'File name'
    column_newname = filename.split(".")[0].lstrip('0').replace('-', '/') 
    doc.columns = [column_newname]
    return doc

In [2]:
doc1 = create_dataframe("01-22-2020.csv")
doc2 = create_dataframe("04-01-2020.csv")

In [3]:
doc2.head()

,4/01/2020
Country_Region,
Afghanistan,237
Albania,259
Algeria,847
Andorra,390
Angola,8


<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">4. Merge Docs</font><br>
</div>

>- For exmaple, merge '01-22-2020.csv' and '04-01-2020.csv'

In [4]:
merge_doc = pd.merge(doc1, doc2, how='outer', left_index=True, right_index=True)
merge_doc.head()

,1/22/2020,4/01/2020
Country_Region,,
Afghanistan,NaN,237
Albania,NaN,259
Algeria,NaN,847
Andorra,NaN,390
Angola,NaN,8


>- Substitute missing values with '0'

In [5]:
merge_doc = merge_doc.fillna(0)
merge_doc

,1/22/2020,4/01/2020
Country_Region,,
Afghanistan,0.0,237
Albania,0.0,259
Algeria,0.0,847
Andorra,0.0,390
Angola,0.0,8
...,...,...
Venezuela,0.0,143
Vietnam,0.0,218
West Bank and Gaza,0.0,134


### <font color='red'>Merge Docs</font>
>- Return the lists of 'csv' files saved in a specific folder
    - os.listdir
>- Sort lists

In [6]:
import os

PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
item_lists = os.listdir(PATH)
csv_list = list()

for item in item_lists:
    if item.split(".")[-1] == 'csv':
        csv_list.append(item)

print(csv_list)

['01-22-2020.csv', '01-23-2020.csv', '01-24-2020.csv', '01-25-2020.csv', '01-26-2020.csv', '01-27-2020.csv', '01-28-2020.csv', '01-29-2020.csv', '01-30-2020.csv', '01-31-2020.csv', '02-01-2020.csv', '02-02-2020.csv', '02-03-2020.csv', '02-04-2020.csv', '02-05-2020.csv', '02-06-2020.csv', '02-07-2020.csv', '02-08-2020.csv', '02-09-2020.csv', '02-10-2020.csv', '02-11-2020.csv', '02-12-2020.csv', '02-13-2020.csv', '02-14-2020.csv', '02-15-2020.csv', '02-16-2020.csv', '02-17-2020.csv', '02-18-2020.csv', '02-19-2020.csv', '02-20-2020.csv', '02-21-2020.csv', '02-22-2020.csv', '02-23-2020.csv', '02-24-2020.csv', '02-25-2020.csv', '02-26-2020.csv', '02-27-2020.csv', '02-28-2020.csv', '02-29-2020.csv', '03-01-2020.csv', '03-02-2020.csv', '03-03-2020.csv', '03-04-2020.csv', '03-05-2020.csv', '03-06-2020.csv', '03-07-2020.csv', '03-08-2020.csv', '03-09-2020.csv', '03-10-2020.csv', '03-11-2020.csv', '03-12-2020.csv', '03-13-2020.csv', '03-14-2020.csv', '03-15-2020.csv', '03-16-2020.csv', '03-17-20

>- lists.sort(): ascending
>- lists.sort(reverse=True): descending

In [7]:
csv_list.sort()
csv_list

['01-22-2020.csv',
 '01-23-2020.csv',
 '01-24-2020.csv',
 '01-25-2020.csv',
 '01-26-2020.csv',
 '01-27-2020.csv',
 '01-28-2020.csv',
 '01-29-2020.csv',
 '01-30-2020.csv',
 '01-31-2020.csv',
 '02-01-2020.csv',
 '02-02-2020.csv',
 '02-03-2020.csv',
 '02-04-2020.csv',
 '02-05-2020.csv',
 '02-06-2020.csv',
 '02-07-2020.csv',
 '02-08-2020.csv',
 '02-09-2020.csv',
 '02-10-2020.csv',
 '02-11-2020.csv',
 '02-12-2020.csv',
 '02-13-2020.csv',
 '02-14-2020.csv',
 '02-15-2020.csv',
 '02-16-2020.csv',
 '02-17-2020.csv',
 '02-18-2020.csv',
 '02-19-2020.csv',
 '02-20-2020.csv',
 '02-21-2020.csv',
 '02-22-2020.csv',
 '02-23-2020.csv',
 '02-24-2020.csv',
 '02-25-2020.csv',
 '02-26-2020.csv',
 '02-27-2020.csv',
 '02-28-2020.csv',
 '02-29-2020.csv',
 '03-01-2020.csv',
 '03-02-2020.csv',
 '03-03-2020.csv',
 '03-04-2020.csv',
 '03-05-2020.csv',
 '03-06-2020.csv',
 '03-07-2020.csv',
 '03-08-2020.csv',
 '03-09-2020.csv',
 '03-10-2020.csv',
 '03-11-2020.csv',
 '03-12-2020.csv',
 '03-13-2020.csv',
 '03-14-2020

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">5. Final Code <font color='red'>(총정리)</font></font><br>
</div>

><b>Step 1. Create Dataframes</b>

In [11]:
import json
import pandas as pd

with open('COVID-19-master/csse_covid_19_data/country_convert.json', 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)

def country_name_convert(arg_df):
    if arg_df['Country_Region'] in json_data:
        arg_df['Country_Region'] = json_data[arg_df['Country_Region']]
    return arg_df['Country_Region']

def create_dataframe(filename):
    PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
    doc = pd.read_csv(PATH + filename, encoding='utf-8-sig')  #1. Read a csv file 
                                                              # filename: e.g., 01-22-2020.csv
    try:
        doc = doc[['Country_Region', 'Confirmed']]  #2. Get a subset: 'Country_Region' & 'Confirmed'
    except:
        doc = doc[['Country/Region', 'Confirmed']]  
        doc.columns = ['Country_Region', 'Confirmed'] #3. Clean messy column names
    doc = doc.dropna(subset=['Confirmed'])     #4. Delete rows with missing values
    doc = doc.astype({'Confirmed': 'int64'})   #5. Change datatype: float -> int
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)   #6. Clean messy country names using 'apply'
    doc = doc.groupby('Country_Region').sum()  #7. Group by 'Country_Region' 
                                               # 'Country_Region' now Index

    #8. Change column name: 'Confirmed' -> 'File name'
    column_newname = filename.split(".")[0].lstrip('0').replace('-', '/') 
    doc.columns = [column_newname]
    return doc

In [9]:
#Test
doc1 = create_dataframe("06-01-2020.csv")
doc1.head()

,6/01/2020
Country_Region,
Afghanistan,15750
Albania,1143
Algeria,9513
Andorra,765
Angola,86


><b>Step 2. Read lists of csv files</b>

In [84]:
import os

PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
item_lists = os.listdir(PATH)
csv_list = list()
    
for item in item_lists:
    if item.split(".")[-1] == 'csv':
        csv_list.append(item)
csv_list.sort()     

><b>Step 3. Merge Dataframes</b>

In [92]:
empty_df = True

for item in csv_list:
    doc = create_dataframe(item)
    if empty_df:  
        empty_df = False
        final_doc = doc     
    else:
        final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)
    
final_doc = final_doc.fillna(0)
final_doc

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,6/08/2020,6/09/2020,6/10/2020,6/11/2020,6/12/2020,6/13/2020,6/14/2020,6/15/2020,6/16/2020,6/17/2020
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20917.0,21459.0,22142.0,22890.0,23546.0,24102.0,24766.0,25527.0,26310.0,26874.0
Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1263.0,1299.0,1341.0,1385.0,1416.0,1464.0,1521.0,1590.0,1672.0,1722.0
Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10265.0,10382.0,10484.0,10589.0,10698.0,10810.0,10919.0,11031.0,11147.0,11268.0
Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,852.0,852.0,852.0,852.0,853.0,853.0,853.0,853.0,854.0,854.0
Angola,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,92.0,96.0,113.0,118.0,130.0,138.0,140.0,142.0,148.0,155.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,0.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,332.0,332.0,332.0,332.0,333.0,334.0,334.0,334.0,334.0,335.0
West Bank and Gaza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,473.0,481.0,485.0,487.0,489.0,489.0,492.0,505.0,514.0,555.0
Yemen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,496.0,524.0,560.0,591.0,632.0,705.0,728.0,844.0,885.0,902.0


><b>Tip. Create Dataframes by PATH</b>

In [94]:
def create_dataframe_by_path(PATH):

    empty_df = True

    for item in csv_list:
        doc = create_dataframe(item)
        if empty_df: #empty_df = True 
            empty_df = False
            final_doc = doc     
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)

    final_doc = final_doc.fillna(0)
    return final_doc

In [98]:
PATH = 'COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/'
doc = create_dataframe_by_path(PATH)

><b>Step 4. Change Datatype and Save as a csv file</b>

In [100]:
doc = doc.astype('int64')
doc.head()

,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,6/08/2020,6/09/2020,6/10/2020,6/11/2020,6/12/2020,6/13/2020,6/14/2020,6/15/2020,6/16/2020,6/17/2020
Country_Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,20917,21459,22142,22890,23546,24102,24766,25527,26310,26874
Albania,0,0,0,0,0,0,0,0,0,0,...,1263,1299,1341,1385,1416,1464,1521,1590,1672,1722
Algeria,0,0,0,0,0,0,0,0,0,0,...,10265,10382,10484,10589,10698,10810,10919,11031,11147,11268
Andorra,0,0,0,0,0,0,0,0,0,0,...,852,852,852,852,853,853,853,853,854,854
Angola,0,0,0,0,0,0,0,0,0,0,...,92,96,113,118,130,138,140,142,148,155


>- Save as a csv file <br>
>    e.g., doc.to_csv('filename')

In [103]:
doc.to_csv('final_covid.csv')